In [ ]:
import collections
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab

In [ ]:
Numb_packets = 100000
Loss_Network = [0.1, 1, 2.5, 5, 10, 20]

#Max Delay
Delays = {}
Delays[0.1] = 8500
Delays[1] = 9000
Delays[2.5] = 9000
Delays[5] = 400000
Delays[10] = 500000
Delays[20] = 700000

#Number of Client Transmissions - Modified QUIC (Limited Retransmissions)
Client_Transmissions_Modified = {}
Client_Transmissions_Modified[0.1] = 100000
Client_Transmissions_Modified[1] = 100000
Client_Transmissions_Modified[2.5] = 100000
Client_Transmissions_Modified[5] = 100000
Client_Transmissions_Modified[10] = 100000
Client_Transmissions_Modified[20] = 100000

#Number of Client Transmissions - Default QUIC
Client_Transmissions = {}
Client_Transmissions[0.1] = 100001
Client_Transmissions[1] = 101207
Client_Transmissions[2.5] = 102073
Client_Transmissions[5] = 105103
Client_Transmissions[10] = 111049
Client_Transmissions[20] = 124758

#Number of Client Transmissions - Improved Default QUIC
Client_Transmissions_Improved = {}
Client_Transmissions_Improved[0.1] = 100000
Client_Transmissions_Improved[1] = 100000
Client_Transmissions_Improved[2.5] = 100000
Client_Transmissions_Improved[5] = 100000
Client_Transmissions_Improved[10] = 100000
Client_Transmissions_Improved[20] = 124984

#Number of Client Transmissions - Improved Modified QUIC
Client_Transmissions_Improved_Modified = {}
Client_Transmissions_Improved_Modified[0.1] = 100000
Client_Transmissions_Improved_Modified[1] = 100000
Client_Transmissions_Improved_Modified[2.5] = 100000
Client_Transmissions_Improved_Modified[5] = 100254
Client_Transmissions_Improved_Modified[10] = 100000
Client_Transmissions_Improved_Modified[20] = 124984

In [ ]:
def GetDelayValues(file_name):
    Delay_Values = {}
    with open(file_name, "r") as file:
        line = file.readline()
        while (line != ""):
            (packet_id, delay_value) = line.split(": ")
            Delay_Values[int(packet_id)] = int(delay_value[:-1])
            line = file.readline()
    return Delay_Values

In [ ]:
def GetOutofOrderPackets(file_name):
    OutofOrderPackets = []
    with open(file_name, "r") as file:
        line = file.readline()
        while (line != ""):
            packet_id = line.split("\n")[0]
            OutofOrderPackets.append(packet_id)
            line = file.readline()
    return OutofOrderPackets

In [ ]:
def GetPacketStats(Delay_Values):
    Bad_Packets = 0
    Good_Packets = 0
    sum_delay_good = 0
    sum_delay_aggregate = 0
    max_delay = 0
    for packet_id in Delay_Values.keys():
        delay = Delay_Values[packet_id]
        sum_delay_aggregate += delay
        if (delay > max_delay):
            max_delay = delay
        if (delay >= 100000):
            Bad_Packets += 1
        else:
            Good_Packets += 1
            sum_delay_good += delay
    return (Bad_Packets, Good_Packets, sum_delay_good, sum_delay_aggregate, max_delay)

In [ ]:
def cdf_delay(data_delay, max_delay, loss):
    ecdf = sm.distributions.ECDF(data_delay)
    delay = np.linspace(min(data_delay), Delays[loss])
    y_delay = ecdf(delay)
    plt.plot(delay, y_delay, color='b', label='One-way Delay')
    plt.grid(True)
    plt.title('Cumulative Distribution Function')
    pylab.legend(loc='lower right')
    plt.ylabel("Cumulative Fraction of Packets")
    plt.xlabel("Delay Value (ms)")
    plt.show()
    return delay, y_delay

def Get95thPercentile(delay, y_delay):
    for i in range(0, len(y_delay)):
        if (y_delay[i] >= 0.95):
            return delay[i]

In [ ]:
def GetData_Total(Loss_Network, path, Client_Transmissions):
    for loss in Loss_Network:
        print("Network loss = " + str(loss) + "%")
        file_name_delay = path + "Loss=" + str(loss) + "/delay_client_server.txt"
        file_name_order = path + "Loss=" + str(loss) + "/Old_Messages.txt"
        Delay_Values = GetDelayValues(file_name_delay)
        print("We received " + str(len(Delay_Values)) + " packets")
        (Bad_Packets, Good_Packets, sum_delay_good, sum_delay_aggregate, max_delay) = GetPacketStats(Delay_Values)
        print("Bad packets:" + str(Bad_Packets) + " packets")
        print("Fraction of good packets (out of the received packets):" + str(Good_Packets/len(Delay_Values) * 100) + "%")
        print("Maximum delay:" + str(max_delay) + " microseconds")
        print("Good average delay:" + str(sum_delay_good/(len(Delay_Values) - Bad_Packets)) + " microseconds")
        print("Aggregate average delay:" + str(sum_delay_aggregate/len(Delay_Values)) + " microseconds")
        
        Loss_rate = Bad_Packets/Numb_packets*100
        print("Loss rate:" + str(Loss_rate) + "%")
        try:
            OutofOrderPackets = GetOutofOrderPackets(file_name_order)
            print("Number of out of order packets:" + str(len(OutofOrderPackets)) + " packets")
        except:
            print("No out of order packets!")
        Retransmissions = Client_Transmissions[loss] - len(Delay_Values)
        print("We record a total of " + str(Retransmissions) + " retransmissions from the client, at loss = " + str(loss) + "%.")
        
        data_delay = np.loadtxt(file_name_delay, usecols=(1))
        #data_delay = data_delay/1000
        (delay, y_delay) = cdf_delay(data_delay, max_delay, loss)
        delay_95 = Get95thPercentile(delay, y_delay)
        print("95'th percentile:" + str(delay_95) + " microseconds\n\n")
        
    return

In [ ]:
def GetData_Modified(Loss_Network, path, Client_Transmissions_Limited):
    for loss in Loss_Network:
        print("Network loss = " + str(loss) + "%")
        file_name_delay = path + "Loss=" + str(loss) + "/delay_client_server.txt"
        file_name_order = path + "Loss=" + str(loss) + "/Old_Messages.txt"
        Delay_Values = GetDelayValues(file_name_delay)
        print("We received " + str(len(Delay_Values)) + " packets")
        (Bad_Packets, Good_Packets, sum_delay_good, sum_delay_aggregate, max_delay) = GetPacketStats(Delay_Values)
        print("Bad packets:" + str(Bad_Packets) + " packets")
        print("Fraction of good packets (out of the received packets):" + str(Good_Packets/len(Delay_Values) * 100) + "%")
        print("Maximum delay:" + str(max_delay) + " microseconds")
        print("Good average delay:" + str(sum_delay_good/(len(Delay_Values) - Bad_Packets)) + " microseconds")
        print("Aggregate average delay:" + str(sum_delay_aggregate/len(Delay_Values)) + " microseconds")
        
        Lost_Packets = Numb_packets - len(Delay_Values)
        print("We record a total of " + str(Lost_Packets) + " lost packets, at loss = " + str(loss) + "%.")
        
        Loss_rate = (Bad_Packets + Lost_Packets)/Numb_packets*100
        print("Loss rate:" + str(Loss_rate) + "%")
        try:
            OutofOrderPackets = GetOutofOrderPackets(file_name_order)
            print("Number of out of order packets:" + str(len(OutofOrderPackets)) + " packets")
        except:
            print("No out of order packets!")
        Retransmissions = Client_Transmissions_Limited[loss] - Numb_packets
        print("We record a total of " + str(Retransmissions) + " retransmissions from the client, at loss = " + str(loss) + "%.")
        
        data_delay = np.loadtxt(file_name_delay, usecols=(1))
        (delay, y_delay) = cdf_delay(data_delay, max_delay, loss)
        delay_95 = Get95thPercentile(delay, y_delay)
        print("95'th percentile:" + str(delay_95) + " microseconds\n\n")
        
    return

In [ ]:
path = "/Users/jstephan/Desktop/Autumn_Semester_2/Semester_Project/Simulations/"
GetData_Total(Loss_Network, path + "Total/", Client_Transmissions)
Loss_Network_test = [5]
GetData_Modified(Loss_Network_test, path + "Improved/Modified/", Client_Transmissions_Improved_Modified)